In [1]:
import ccxt
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go
import numpy as np
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import praw
import datetime
import time
import re
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
from textblob import TextBlob
from collections import Counter

# LPPL model on Dogecoin

In [2]:
exchange = ccxt.binance()
markets = exchange.load_markets()
DOGE_markets = [market for market in markets if 'DOGE' in market]

DOGE_markets

['DOGE/BNB',
 'DOGE/BTC',
 'DOGE/USDT',
 'DOGE/PAX',
 'DOGE/USDC',
 'DOGE/BUSD',
 'DOGE/EUR',
 'DOGE/TRY',
 'DOGE/AUD',
 'DOGE/BRL',
 'DOGE/GBP',
 'DOGE/RUB',
 'DOGE/BIDR',
 'SHIB/DOGE',
 'DOGE/TUSD',
 'DOGE/FDUSD',
 'DOGE/JPY',
 '1MBABYDOGE/USDT',
 '1MBABYDOGE/FDUSD',
 '1MBABYDOGE/TRY',
 '1MBABYDOGE/USDC',
 'DOGE/USDT:USDT',
 'DOGE/USDC:USDC',
 '1MBABYDOGE/USDT:USDT',
 'DOGE/USD:DOGE']

In [3]:
symbol = 'DOGE/USDT'
timeframe = '1h'
limit_per_request = 1000

In [5]:
start_date = datetime.datetime(2024, 8, 4, 19, 0)
end_date = datetime.datetime(2025, 1, 31, 18, 0)


start_timestamp = int(start_date.timestamp() * 1000)
end_timestamp = int(end_date.timestamp() * 1000)

all_ohlcv = []
current_timestamp = start_timestamp

while current_timestamp < end_timestamp:
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=current_timestamp, limit=limit_per_request)

        if not ohlcv:
            print(f"No more data available after {datetime.utcfromtimestamp(current_timestamp / 1000)}")
            break

        all_ohlcv.extend(ohlcv)

        
        current_timestamp = ohlcv[-1][0]

        
        exchange.sleep(1000)

    except Exception as e:
        print(f"Error fetching data: {e}")
        break


df = pd.DataFrame(all_ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"])

df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")


df = df[(df["timestamp"] >= start_date) & (df["timestamp"] <= end_date)]

df = df.reset_index(drop=True)

df.index = range(len(df)) 


In [6]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df = df.drop_duplicates(subset=['timestamp'])
df = df.sort_values(by='timestamp').reset_index(drop=True)
df

,timestamp,open,high,low,close,volume
0,2024-08-04 19:00:00,0.10308,0.10542,0.10215,0.10479,61252561.0
1,2024-08-04 20:00:00,0.10479,0.10600,0.10412,0.10497,33856617.0
2,2024-08-04 21:00:00,0.10497,0.10516,0.10432,0.10463,24099918.0
3,2024-08-04 22:00:00,0.10463,0.10598,0.10417,0.10520,38280198.0
4,2024-08-04 23:00:00,0.10519,0.10535,0.10357,0.10375,38682391.0
...,...,...,...,...,...,...
4315,2025-01-31 14:00:00,0.33569,0.33791,0.33058,0.33242,108335653.0
4316,2025-01-31 15:00:00,0.33243,0.34177,0.33242,0.33952,92216402.0
4317,2025-01-31 16:00:00,0.33952,0.34100,0.33574,0.33824,60499525.0
4318,2025-01-31 17:00:00,0.33825,0.33838,0.33299,0.33301,30400286.0


In [7]:
df = df.sort_values(by='timestamp').reset_index(drop=True)
df
df.to_csv("doge_1h_gran_6m.csv", index=False)

In [8]:
transparent_template = go.layout.Template()
transparent_template.layout.paper_bgcolor = 'rgba(0,0,0,0)'  # Transparent outer background
transparent_template.layout.plot_bgcolor = 'rgba(0,0,0,0)' 

custom_title = "DOGE/USDT Hourly Price Evolution (<span style='color:green;'>increasing</span>/<span style='color:red;'>decreasing</span> in the last 6 months)"

fig = go.Figure(data=[go.Candlestick(
    x=df['timestamp'],  
    open=df['open'],    
    high=df['high'],    
    low=df['low'],      
    close=df['close'],  
    increasing_line_color='green',  
    decreasing_line_color='red'     
)])


fig.update_layout(
    title=dict(
        text=custom_title
    ),
    xaxis_title='Time',
    yaxis_title='Price (USDT)',
    xaxis_rangeslider_visible=False,  
    template=transparent_template,           
    hovermode='x unified',
    showlegend=False              
)

fig.add_trace(go.Bar(
    x=df['timestamp'],
    y=df['volume'],
    name='Volume',
    marker_color='rgba(150, 150, 150, 0.8)', 
    yaxis='y2' 
))


fig.update_layout(
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False  
    )
)

fig.show()